# Converting the Belgium cells from original csv file into an importable circus actor.  

## Parsing raw file

In [1]:
import pandas as pd

cells = pd.read_csv("cells.csv")
cells.site_name = cells.site_name.str.upper().str.replace(" ", "_")

# removing the 3 random sites 
cells = cells[~(cells.site_name.str[:11] == "SITE_RANDOM")]

sites = cells[["site_name", "site_longitude", "site_latitude", "geo_level1_id", "site_urban"]].drop_duplicates()

sites.shape

(4208, 5)

In [5]:
sites.sample(8)

,site_name,site_longitude,site_latitude,geo_level1_id,site_urban
41400,SITE_005313,4.480148,51.217945,LVL1_Anvers,True
49458,SITE_003372,4.734956,51.076495,LVL1_Heist-op-den-Berg,False
45078,SITE_002938,4.522594,50.887851,LVL1_Kortenberg,True
14988,SITE_001832,4.385839,51.206518,LVL1_Anvers,True
47058,SITE_009888,3.411210,50.627120,LVL1_Tournai,False
36990,SITE_006338,5.512299,50.872019,LVL1_Bilzen,False
49020,SITE_001632,3.354240,50.842673,LVL1_Deerlijk,False
48306,SITE_011141,4.403788,50.874687,LVL1_Evere,True


## Converting to Actor

In [6]:
from datagenerator.core import actor
from datagenerator.core import circus

circus = circus.Circus(name="belgium", master_seed=1234, 
                start=pd.Timestamp("22 July 2012"), 
                step_duration=pd.Timedelta("12h") )
sites_actor = circus.create_actor(name="sites", ids=sites.site_name)

sites_actor.create_attribute(name="LATITUDE",    init_values=sites.site_latitude );
sites_actor.create_attribute(name="LONGITUDE",   init_values=sites.site_longitude);
sites_actor.create_attribute(name="GEO_LEVEL_1", init_values=sites.geo_level1_id );
sites_actor.create_attribute(name="URBAN", init_values=sites.site_urban );

cell_rel = sites_actor.create_relationship(name="CELLS")
cell_rel.add_relations(from_ids=cells.site_name,to_ids=cells.cell_id)


In [7]:
circus.actors["sites"].to_dataframe().sample(4)

,LATITUDE,URBAN,GEO_LEVEL_1,LONGITUDE
site_name,,,,
SITE_009093,50.463065,False,LVL1_Namur,4.865139
SITE_004610,51.110042,False,LVL1_Oostkamp,3.184321
SITE_004380,50.663254,False,LVL1_Silly,3.935571
SITE_016659,50.846150,True,LVL1_Bruxelles,4.386202


## Saving

In [8]:
circus.save_to_db(overwrite=True)